In [80]:
import json
import csv
import os
from glob import glob
import random
import re

#Rutas donde se almacenaran los archivos
folder_path = "CorpusBERT/"
qa_corpus_name = "trainQACorpus.json"
pre_trainer_corpus_name = "trainPreTrainerCorpus.json"
test_qa_corpus_name = "testQACorpus.json"
test_pre_trainer_corpus_name = "testPreTrainerCorpus.json"
os.makedirs(os.path.dirname(folder_path), exist_ok=True)

In [73]:
# Se tienen 2 fuentes de datos

# Corpus base con definiciones clave
base_corpus = [
    ("¿Qué es un Feature Group?", "Un Feature Group es una colección de variables utilizadas en modelos analíticos dentro del Feature Store del banco."),
    ("¿Qué significa <vp>?", "<vp> representa la vicepresidencia a la que pertenece el Feature Group, puede ser 'per' para Banca Persona o 'emp' para Banca Empresa."),
    ("¿Qué significa <lat>?", "<lat> indica la periodicidad de carga del Feature Group, puede ser 'dia' (Diario), 'sem' (Semanal) o 'mes' (Mensual)."),
    ("¿Qué es el universo máximo en los datos del banco?", "El universo máximo define el conjunto de datos sobre el cual se aplican los Feature Groups: Población completa, Sistema financiero o Clientes del banco."),
]

# DEFINICIÓN-DE-DOMINIOS-Y-FEATURE-GROUPS-EN-EL-FEATURE-STORE data
json_data = [
    ["Información personas ","ibk-fs-acoe-infpers","Datos sociodemográficos","<vp>-sociodem-<lat>","ibk_fs_group_<vp>_sociodem_<lat>"," Población completa","Información sociodemográfica, ingresos, edad, años de existencia y fuentes de ingresos."],
    ["Información personas ","ibk-fs-acoe-infpers","Propiedades ","<vp>-propiedad-<lat>","ibk_fs_group_<vp>_propiedad_<lat>"," Población completa","Información de propiedades muebles e inmuebles, tangibles e intangibles."],
    ["Información personas ","ibk-fs-acoe-infpers","Datos de contacto ","<vp>-contacto-<lat>","ibk_fs_group_<vp>_contacto_<lat>"," Población completa","Contactabilidad a nivel de personas para teléfonos o correos. Incluye la cantidad o ratios de tipos de contactos, tales como el contacto efectivo, contacto no efectivo o sin contacto, para diferentes periodos.  "],
    ["Información personas ","ibk-fs-acoe-infpers","Contacto telefónico","ibk-ctostelf-<lat>","ibk_fs_group_ibk_ctostelf<lat>","Teléfonos de la población completa","Contactabilidad a nivel de teléfonos, no a nivel de personas. Incluye la cantidad o ratios de tipos de contactos, tales como el contacto efectivo, contacto no efectivo o sin contacto, para diferentes periodos."],
    ["Información personas ","ibk-fs-acoe-infpers","Contacto digital","ibk-ctosdig-<lat>","ibk_fs_group_ibk_ctosdig_<lat>","Correos electrónicos de la población completa","Contactabilidad a nivel de correos electrónicos, no a nivel de personas. Incluye la cantidad o ratios de tipos de contactos, tales como el contacto efectivo, contacto no efectivo o sin contacto, para diferentes periodos.  "],
    ["Historial financiero ","ibk-fs-acoe-histfin","RCC ","<vp>-rcc-<lat>","ibk_fs_group_<vp>_rcc_<lat>","Sistema financiero","Información sobre los productos financieros, montos y variaciones en los montos de productos activos reportados en el sistema financiero."],
    ["Historial financiero ","ibk-fs-acoe-histfin","Riesgo financiero ","<vp>-rskfinac-<lat>","ibk_fs_group_<vp>_rskfinac_<lat>","Sistema financiero","Variables relacionadas con la evaluación del riesgo crediticio, días de mora o atraso en todo el sistema financiero (no exclusivamente Interbank) e información de centrales de riesgo de pagos."],
    ["Información cliente ","ibk-fs-acoe-infocli","Perfil del cliente ","<vp>-pflcom-<lat>","ibk_fs_group_<vp>_pflcom_<lat>","Sistema financiero","Descripciones sobre el perfil comercial de los potenciales clientes del banco, tales como el CLV del cliente, principalidad; y, segmentaciones o perfilamientos sobre el tipo de consumidor o nivel de digitalización, etc."],
    ["Información cliente ","ibk-fs-acoe-infocli","Bases de riesgos ibk","<vp>-rskibk-<lat>","ibk_fs_group_<vp>_rskibk_<lat>"," Sistema financiero","Información sobre los productos financieros aprobados por Interbank a clientes y no clientes, sin considerar si son ofertados o no. Incluye el monto ofertado, la tasa y la capacidad de endeudamiento máxima, conocida como CEM. Además, la evaluación del riesgo crediticio, días de mora o atraso en exclusivamente Interbank."],
    ["Información cliente ","ibk-fs-acoe-infocli","Productos ","<vp>-products-<lat>","ibk_fs_group_<vp>_products_<lat>"," Clientes del banco","Descripción de los montos, duración del préstamo, períodos de gracia, etc. Además de la cantidad de productos (activos y pasivos) y el saldo de productos."],
    ["Gestión del journey ","ibk-fs-acoe-journey","Journey ","<vp>-journey-<lat>","ibk_fs_group_<vp>_journey_<lat>"," Clientes del banco","Información sobre las etapas del ciclo de vida del cliente, tales como la adquisición, activación, desarrollo o retención."],
    ["Gestión del journey ","ibk-fs-acoe-journey","Churn ","<vp>-churn-<lat>","ibk_fs_group_<vp>_churn_<lat>"," Clientes del banco","Variables sobre los productos cancelados o bloqueados, tipos de cancelaciones, intentos de cancelaciones y canal de cancelación."],
    ["Campañas y promociones ","ibk-fs-acoe-camp","Oferta de productos ","<vp>-cmpprod-<lat>","ibk_fs_group_<vp>_cmpprod_<lat>"," Sistema financiero","Datos relacionados con la participación del cliente en campañas de oferta de productos, los productos ofertados, respuestas y canales."],
    ["Campañas y promociones ","ibk-fs-acoe-camp","Campañas de consumo ","<vp>-cmpcons-<lat>","ibk_fs_group_<vp>_cmpcons_<lat>"," Clientes del banco","Datos relacionados con la participación del cliente en campañas de consumo con crédito o débito, cantidad o ratios de promociones enviadas, tipos de promociones, canales de envío, nivel de recepción y respuestas."],
    ["Campañas y promociones ","ibk-fs-acoe-camp","Campañas de educación ","<vp>-cmpedu-<lat>","ibk_fs_group_<vp>_cmpedu_<lat>"," Clientes del banco","Datos relacionados con la participación del cliente en campañas de educación de uso de productos o app, cantidad o ratios de comunicación enviadas, canales de envío, nivel de recepción y respuestas."],
    ["Transacciones financieras ","ibk-fs-acoe-trx","Transacciones ","<vp>-trx-<lat>","ibk_fs_group_<vp>_trx_<lat>"," Clientes del banco","Información sobre las transacciones realizadas por los clientes, montos transaccionados, medios y canales por donde se transacciono."],
    ["Transacciones financieras ","ibk-fs-acoe-trx","Consumos ","<vp>-consumos-<lat>","ibk_fs_group_<vp>_consumos_<lat>"," Clientes del banco","Información de los consumos con tarjetas realizadas por los clientes, incluyendo el rubro de consumo y medio o canal."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias de en tiendas ","<vp>-exptienda-<lat>","ibk_fs_group_<vp>_exptienda_<lat>"," Sistema financiero","Información de la interacción de los clientes en las tiendas, sus arribos, NPS, tiempos de espera, motivos de arribos, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias televentas ","<vp>-exptlv-<lat>","ibk_fs_group_<vp>_exptlv_<lat>"," Sistema financiero","Información de la interacción de los clientes y potenciales clientes en televentas, motivos de no aceptación, cantidad de llamadas, productos ofertados por canal, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias banca telefónica ","<vp>-bnktelf-<lat>","ibk_fs_group_<vp>_bnktelf_<lat>"," Sistema financiero","Información de la interacción de los clientes o potenciales clientes en banca telefónica, motivos de llamada, cantidad de llamadas, tiempos de respuesta, cadencia, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias chats ","<vp>-expchats-<lat>","ibk_fs_group_<vp>_expchats_<lat>"," Sistema financiero","Información de la comunicación de los clientes o potenciales clientes con los chatbots u operadores de chats (whatsapp, Facebook, etc). Incluyendo los motivos de la comunicación, resolutividad, tiempos de respuesta, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencia APP ","<vp>-expapp-<lat>","ibk_fs_group_<vp>_expapp_<lat>"," Clientes del banco","Información de experiencia de los clientes con las apps del banco, incluyendo tiempos de navegación, clicks, número de consultas en la app, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencia digital ","<vp>-expdig-<lat>","ibk_fs_group_<vp>_expdig_<lat>"," Sistema financiero","Información sobre otras variables digitales de los clientes que no tengan que ver con transacciones o las apps del banco, incluyendo uso de canales físicos-digitales, como ATMs y variables de la web del banco."],
    ["Cobranzas ","ibk-fs-acoe-cob","Comportamiento de pagos","<vp>-pagos-<lat>","ibk_fs_group_<vp>_pagos_<lat>"," Clientes del banco","Historial de pagos por tipo de producto dentro de Interbank, incluyendo días de mora y deuda acumulada."],
    ["Cobranzas ","ibk-fs-acoe-cob","Medios de cobranza","<vp>-cobros-<lat>","ibk_fs_group_<vp>_cobros_<lat>"," Clientes del banco","Medios de comunicación para las cobranzas, cantidad de llamadas, cantidad de llamadas rechazadas, etc."],
    ["Variables INTERCORP","ibk-fs-acoe-intercorp","Intercorp transformada","ibk-itcx-<lat>","ibk_fs_group_ibk_itcx_<lat>","Población completa","Información sobre las variables transformadas de intercorp, y en que modelos se utilizan."]                            
]

# meta_fs - corpus.csv
csv_path = folder_path + "/meta_fs - corpus.csv"
csv_data = []
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None) #OBviamos los headers
    for row in reader:
        processed_row = [col if col.strip() else "nan" for col in row]
        csv_data.append(processed_row)

In [74]:
#Save data Funcion
import json
import os

def save_data(file_path, new_items):
    if os.path.exists(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                existing_data = json.load(f)
                if not isinstance(existing_data, list):  
                    existing_data = []
        except (json.JSONDecodeError, FileNotFoundError):
            existing_data = [] 
    else:
        existing_data = []

    existing_data.extend(new_items)
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)


In [75]:
def process_base_corpus(base_corpus, folder_path, qa_corpus_name):
    save_data(folder_path + qa_corpus_name, base_corpus)

In [76]:
def process_json_corpus(data, folder_path, qa_corpus_name, pre_trainer_corpus_name): 
    qa_items = []
    preTrainer_items = []   
    for row in data:
        dominio, repo, feature_group, bitbucket_fg, aws_tabla, universo, definicion = row
        
        # Pares de pregunta-respuesta
        qa_items.extend([
            (f"¿Qué información contiene el feature group '{feature_group.strip()}'?", definicion.strip()),
            (f"¿Dónde se almacena el feature group '{feature_group.strip()}'?", f"Repositorio: {repo.strip()}, Bitbucket: {bitbucket_fg.strip()}, AWS Tabla: {aws_tabla.strip()}"),
            (f"¿Cuál es el universo de datos del feature group '{feature_group.strip()}'?", universo.strip()),
            (f"¿Cómo está relacionado el feature group '{feature_group.strip()}' con el dominio '{dominio.strip()}'?", f"Forma parte del dominio {dominio.strip()} y se encuentra en el repositorio {repo.strip()}."),
            (f"¿Cuál es el propósito del feature group '{feature_group.strip()}' en el contexto del banco?", f"{definicion.strip()} Este feature group pertenece al dominio {dominio.strip()} y su universo máximo es {universo.strip()}."),
        ])
        
        # Texto estructurado
        structured_text = (
            f"Dominio: {dominio.strip()}\n"
            f"Repositorio: {repo.strip()}\n"
            f"Feature Group: {feature_group.strip()}\n"
            f"Bitbucket Feature Group: {bitbucket_fg.strip()}\n"
            f"AWS Tabla: {aws_tabla.strip()}\n"
            f"Universo Máximo: {universo.strip()}\n"
            f"Definición: {definicion.strip()}\n"
        )
        preTrainer_items.append(structured_text)

    save_data(folder_path + qa_corpus_name, qa_items)
    save_data(folder_path + pre_trainer_corpus_name, preTrainer_items)

In [77]:
def process_csv_corpus(data, folder_path, qa_corpus_name, pre_trainer_corpus_name): 
    qa_items = []
    preTrainer_items = []
    for row in data:
        variable, descripcion, tablas_fuente, dominio, p_featuregroup = row
        # Pares de pregunta-respuesta
        qa_items.extend([
            (f"¿Qué significa la variable '{variable.strip()}'?", descripcion.strip()),
            (f"¿A qué tabla fuente pertenece la variable '{variable.strip()}'?", f"Se encuentra en la tabla fuente: {tablas_fuente.strip()}"),
            (f"¿A qué dominio pertenece la variable '{variable.strip()}'?", f"Forma parte del dominio: {dominio.strip()}"),
            (f"¿En qué feature group se encuentra la variable '{variable.strip()}'?", f"Se encuentra en el feature group: {p_featuregroup.strip()}"),
            (f"¿Cuál es el propósito de la variable '{variable.strip()}'?", f"{descripcion.strip()} Esta variable pertenece al dominio {dominio.strip()} y está en la tabla {tablas_fuente.strip()} dentro del feature group {p_featuregroup.strip()}.")
        ])
        
        # Texto estructurado
        structured_text = (
            f"Variable: {variable.strip()}\n"
            f"Descripción: {descripcion.strip()}\n"
            f"Tabla Fuente: {tablas_fuente.strip()}\n"
            f"Dominio: {dominio.strip()}\n"
            f"Feature Group: {p_featuregroup.strip()}\n"
        )
        preTrainer_items.append( structured_text)

    save_data(folder_path + qa_corpus_name, qa_items)
    save_data(folder_path + pre_trainer_corpus_name, preTrainer_items)

In [78]:
#eliminanos archivos anteriores
if os.path.exists(folder_path + qa_corpus_name):
    os.remove(folder_path + qa_corpus_name)
if os.path.exists(folder_path + pre_trainer_corpus_name):
    os.remove(folder_path + pre_trainer_corpus_name)

process_base_corpus(base_corpus, folder_path, qa_corpus_name)
process_json_corpus(json_data, folder_path, qa_corpus_name, pre_trainer_corpus_name)
process_csv_corpus(csv_data, folder_path, qa_corpus_name, pre_trainer_corpus_name)

In [ ]:
#GENERATE TEST CORPUS FOR MASKED PRETRAINER

# Diccionario de sinónimos o variaciones
replacement_dict = {
    r"\bDominio\b": ["Ámbito", "Categoría", "Sector"],
    r"\bRepositorio\b": ["Fuente de datos", "Almacén de información"],
    r"\bFeature Group\b": ["Grupo de atributos", "Conjunto de variables"],
    r"\bBitbucket Feature Group\b": ["Repositorio de features", "Grupo de código"],
    r"\bAWS Tabla\b": ["Tabla en AWS", "Base de datos en la nube"],
    r"\bUniverso Máximo\b": ["Cobertura total", "Población completa"],
    r"\bDefinición\b": ["Explicación", "Concepto"],
    r"\bDescripción\b": ["Detalle", "Explicación breve"],
    r"\bVariable\b": ["Parámetro", "Atributo"],
    r"\bIndicador\b": ["Flag", "Señal", "Estado"],
    r"\bTabla Fuente\b": ["Tabla Base", "Dataset Origen", "Fuente primaria"],
    r"\bPoblación completa\b": ["Todos los clientes", "Usuarios activos"],
    r"\bt_agg_cambiofecha_hist_rsk\b": ["t_cambio_fecha_hist", "hist_cambios_fecha"]
}

# Función para generar errores tipográficos aleatorios
def introduce_typo(word):
    if len(word) > 3 and random.random() < 0.3:  # 30% de probabilidad de error
        typo_type = random.choice(["swap", "delete", "duplicate"])
        idx = random.randint(0, len(word) - 2)
        if typo_type == "swap":
            word = word[:idx] + word[idx + 1] + word[idx] + word[idx + 2:]  # Intercambia letras
        elif typo_type == "delete":
            word = word[:idx] + word[idx + 1:]  # Elimina una letra
        elif typo_type == "duplicate":
            word = word[:idx] + word[idx] + word[idx] + word[idx + 1:]  # Duplica una letra
    return word

# Función para generar un corpus de prueba
def generate_test_corpus(train_corpus, replacement_dict, test_ratio=1.0, introduce_errors=True):
    selected_samples = random.sample(train_corpus, int(len(train_corpus) * test_ratio)) if test_ratio < 1 else train_corpus
    test_corpus = []

    for text in selected_samples:
        modified_text = text  # No usar strip() para preservar saltos de línea y espacios
        
        # Aplicar reemplazos según el diccionario
        for pattern, variations in replacement_dict.items():
            modified_text = re.sub(pattern, random.choice(variations), modified_text)

        # Introducir errores tipográficos en palabras aleatorias
        if introduce_errors:
            words = modified_text.split()
            words = [introduce_typo(word) if random.random() < 0.2 else word for word in words]  # 20% de palabras con error
            modified_text = " ".join(words)
        
        test_corpus.append(modified_text)
    
    return test_corpus

# Leer corpus de entrenamiento desde un archivo
with open(folder_path + pre_trainer_corpus_name, "r", encoding="utf-8") as f:
    train_corpus = f.readlines()  # Mantiene saltos de línea

# Configuración del tamaño del corpus de prueba
TEST_RATIO = 0.3  # 30% del tamaño del corpus de entrenamiento

# Generar corpus de prueba
test_corpus = generate_test_corpus(train_corpus, replacement_dict, test_ratio=TEST_RATIO)

# Guardar en un archivo JSON correctamente formateado
with open(folder_path + test_pre_trainer_corpus_name, "w", encoding="utf-8") as f:
    json.dump(test_corpus, f, ensure_ascii=False, indent=4)  # Se mantiene el formato JSON

with open(folder_path + test_pre_trainer_corpus_name, "r", encoding="utf-8") as f:
    json_text = f.read()

# Restaurar "\n" como texto simple sin interpretarlo
json_text = json_text.replace("\\n", "n")
json_text = json_text.replace("\n", " \n")
json_text = json_text.replace("\\\"", "")

# Guardar nuevamente como texto simple sin modificar la estructura JSON
with open(folder_path + test_pre_trainer_corpus_name, "w", encoding="utf-8") as f:
    f.write(json_text)

print(f"✅ Corpus de prueba generado exitosamente con {len(test_corpus)} muestras en {folder_path + test_pre_trainer_corpus_name}")

✅ Corpus de prueba generado exitosamente con 332 muestras en CorpusBERT/testPreTrainerCorpus.json


In [81]:
#GENERATE TEST CORPUS FOR QA TRAINER

# Función para generar preguntas reformuladas
def generar_pregunta_simulada(pregunta):
    variaciones = [
        f"¿Podrías explicarme {pregunta[2:].lower()}?", 
        f"Me gustaría saber {pregunta[2:].lower()}.",
        f"Tengo dudas sobre {pregunta[2:].lower()}, ¿puedes ayudarme?",
        f"¿Cómo se define {pregunta[2:].lower()}?",
        f"¿Qué significa exactamente {pregunta[2:].lower()}?"
    ]
    return random.choice(variaciones)

#Se crea a aprtir del conjunto de entrenamiento
with open(folder_path + qa_corpus_name, "r", encoding="utf-8") as file:
    corpus = json.load(file)

preguntas_respuestas = [pair for pair in corpus if pair[0].startswith("¿")]
test_set_size = int(len(preguntas_respuestas) * 0.2)
test_set_simulated = [
    [generar_pregunta_simulada(q), a] for q, a in random.sample(preguntas_respuestas, test_set_size)
]

# Guardar el conjunto de prueba generado
with open(folder_path + test_qa_corpus_name, "w", encoding="utf-8") as file:
    json.dump(test_set_simulated, file, ensure_ascii=False, indent=4)